# LinearRegression type

- Properties declared in a struct
- Function that initializes regression (analogous to constructor)
- predict and loss functions/methods.

In [1]:
import LinearAlgebra
const LA = LinearAlgebra

LinearAlgebra

In [2]:
mutable struct LinearRegression
    weights::Matrix
    bias::Float64
    name::String
end

In [3]:
LinearRegression(nparams, name) = LinearRegression(randn(1, nparams), 0.0, name)

LinearRegression

In [4]:
function predict(model::LinearRegression, X)
    model.weights * X .+ model.bias
end

predict (generic function with 1 method)

In [5]:
function loss(model::LinearRegression, X, y)
    LA.norm(predict(model, X) .- y, 2)
end

loss (generic function with 1 method)

## Simulated Data

Generate some values from ground truth coefficients and bias, noised.

In [6]:
weights_gt = [1.0, 2.7, 0.3, 1.2]'
bias_gt = 0.4

X = randn(length(weights_gt), 10000)
y = weights_gt * X .+ bias_gt

X .+= 0.001 .* randn(size(X))

4×10000 Array{Float64,2}:
 -1.42377    0.684539   0.3915    …  -1.35926    0.575699    0.0628009
 -0.914241   0.516112   0.988799      0.252688   0.767532   -1.31141  
  0.967221  -0.533801   0.425579     -1.90016   -0.0081994   0.167494 
 -0.429949  -1.08626   -0.545672     -0.063323   0.0632546  -0.118959 

In [7]:
import Zygote

In [8]:
model = LinearRegression(size(X, 1), "Example")

grads = Zygote.gradient(model) do m
    return loss(m, X[:,1], y[1])
end

(Base.RefValue{Any}((weights = [-1.423771882374618 -0.9142405236059253 0.9672213266956339 -0.4299485194248357], bias = 1.0, name = nothing)),)

In [11]:
grads[1]

Base.RefValue{Any}((weights = [-1.423771882374618 -0.9142405236059253 0.9672213266956339 -0.4299485194248357], bias = 1.0, name = nothing))

In [12]:
grads[1][]

(weights = [-1.423771882374618 -0.9142405236059253 0.9672213266956339 -0.4299485194248357], bias = 1.0, name = nothing)

In [13]:
function sgd_update!(model::LinearRegression, grads, η = 0.001)
    model.weights .-= η .* grads.weights
    model.bias -= η * grads.bias
end

sgd_update! (generic function with 2 methods)

In [16]:
@info("Running train loop for $(size(X,2)) iterations")
for idx in 1:size(X, 2)
    grads = Zygote.gradient(m -> loss(m, X[:, idx], y[idx]), model)[1][]
    sgd_update!(model, grads)
end

┌ Info: Running train loop for 10000 iterations
└ @ Main In[16]:1


In [17]:
weights_gt

1×4 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.0  2.7  0.3  1.2

In [18]:
model.weights

1×4 Array{Float64,2}:
 1.00592  2.69935  0.300931  1.19765